In [ ]:
# Import the functions 
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import curve_fit
from sim_POMDP_reaction_RDM_func import sim_POMDP_reaction_RDM_func

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Parameters
std_i = 0.46
w_z = 0.52
std_z = 2.5
learning_rate = 0.15
reward = [1, -1]
termination_thres = 0.0005
termination_factor = 0.5
conf_thres = 0.58
trial_num = 2200
max_t = 100
iter_num = 100

# Load data (you'll need to replace this with your actual data loading)
data = {
    'scoh': np.random.choice([-0.512, -0.256, -0.128, -0.064, -0.032, 0, 0.032, 0.064, 0.128, 0.256, 0.512], size=trial_num),
    'direction': np.random.choice([0, 1], size=trial_num)
}

# Run simulation
simulation = sim_POMDP_reaction_RDM_func(std_i, w_z, std_z, learning_rate, reward,
                                         termination_thres, termination_factor, conf_thres,
                                         trial_num, max_t, iter_num, data)

# Extract results
true_coh_total = simulation['true_coh_total']
action_total = simulation['action_total']
reward_total = simulation['reward_total']
observe = simulation['observe']
conf_total = simulation['conf_total']
RPE_total = simulation['RPE_total']

# Calculate average reward
mean_reward = np.mean(reward_total, axis=1)

# Plot learning curve
plt.figure(figsize=(10, 6))
plt.plot(mean_reward)
plt.xlabel('Episode')
plt.ylabel('Average reward')
plt.title('Learning Curve')
plt.xlim([-100, 5000])
plt.show()

# Calculate psychometric curve
start_idx = trial_num // 3
plot_coh = true_coh_total[start_idx:].flatten()
plot_action = action_total[start_idx:].flatten()

cohs = np.unique(plot_coh)
perc_right = np.array([np.mean(plot_action[plot_coh == coh]) for coh in cohs])
n = np.array([np.sum(plot_coh == coh) for coh in cohs])
perc_right_SE = np.sqrt(perc_right * (1 - perc_right) / n)

# Fit logistic regression
def logistic(x, x0, k):
    return 1 / (1 + np.exp(-k * (x - x0)))

popt, _ = curve_fit(logistic, cohs, perc_right)

# Plot psychometric curve
plt.figure(figsize=(10, 6))
plt.errorbar(cohs * 100, perc_right, yerr=perc_right_SE, fmt='o')
plt.plot(np.linspace(cohs[0], cohs[-1], 100) * 100, logistic(np.linspace(cohs[0], cohs[-1], 100), *popt), 'r-')
plt.xlabel('Motion strength (Coh %)')
plt.ylabel('Proportion rightward choices')
plt.title('Psychometric Curve')
plt.show()

# Calculate reaction time
RT = np.sum(observe, axis=0)
meanRT = np.array([np.mean(RT[plot_coh == coh]) for coh in cohs])
seRT = np.array([np.std(RT[plot_coh == coh]) / np.sqrt(np.sum(plot_coh == coh)) for coh in cohs])

# Plot reaction time
plt.figure(figsize=(10, 6))
plt.errorbar(cohs * 100, meanRT, yerr=seRT, fmt='o-')
plt.xlabel('Motion strength (Coh %)')
plt.ylabel('Reaction time (Time step)')
plt.title('Reaction Time vs. Coherence')
plt.show()

# Calculate confidence
plot_conf = conf_total[start_idx:].flatten()
mean_conf = np.array([np.mean(plot_conf[plot_coh == coh]) for coh in cohs])
std_conf = np.array([np.std(plot_conf[plot_coh == coh]) / np.sqrt(np.sum(plot_coh == coh)) for coh in cohs])

# Plot confidence
plt.figure(figsize=(10, 6))
plt.errorbar(cohs * 100, mean_conf, yerr=std_conf, fmt='o-')
plt.xlabel('Motion strength (Coh %)')
plt.ylabel('Confidence')
plt.title('Confidence vs. Coherence')
plt.show()

# Calculate RPE
plot_RPE = RPE_total[start_idx:].flatten()
mean_RPE = np.array([np.mean(plot_RPE[(plot_coh == coh) & (reward_total[start_idx:].flatten() > 0)]) for coh in cohs])
std_RPE = np.array([np.std(plot_RPE[(plot_coh == coh) & (reward_total[start_idx:].flatten() > 0)]) / 
                    np.sqrt(np.sum((plot_coh == coh) & (reward_total[start_idx:].flatten() > 0))) for coh in cohs])

# Plot RPE
plt.figure(figsize=(10, 6))
plt.errorbar(cohs * 100, mean_RPE, yerr=std_RPE, fmt='o-')
plt.xlabel('Motion strength (Coh %)')
plt.ylabel('Reward Prediction Error')
plt.title('RPE vs. Coherence (Correct Trials)')
plt.show()
